Author: <font color="blue">Miguel Rando Ramirez</font>

E-mail: <font color="blue">miguel.rando01@estudiant.upf.edu</font>

Date: <font color="blue">26/11/2024</font>

In [1]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip

# 0. Dataset and how to iterate

In [2]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

In [3]:
# Leave this code as-is

POS_NOUN = 'NN'
POS_VERB = 'VB'
POS_ADJECTIVE = 'JJ'

# Producer in Python that reads a file by words that are nouns
def read_by_parts_of_speech(filename, parts_of_speech, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for sentence in nltk.sent_tokenize(text):
                
                tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
                for word in [part[0] for part in tagged if part[1] in parts_of_speech]:
                
                    counter += 1

                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [6]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/miguel/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
for word in read_by_parts_of_speech(INPUT_FILE, [POS_ADJECTIVE], max_words=30000, report_every=10000):
    # Prints 1/1000 of words
    if random.random() < 0.001:
        print("Current noun '%s'" % (word)) 

Current noun 'same'
Current noun 'ta'
Current noun 'few'
Current noun 'such'
Current noun 'dear'
Current noun 'many'
Current noun 'liberty'
Current noun 'right'
Current noun 'numb'
Current noun 'good'
Current noun 'schizophrenic'
Current noun 'few'
Current noun 'many'
Current noun 'dumb'
- Read 10000/30000 words so far
Current noun 'detective'
Current noun 'unknown'
Current noun 'eventual'
Current noun 'regrettable'
Current noun 'glad'
Current noun 'future'
Current noun 'short'
Current noun 'british'
Current noun 'new'
Current noun 'first'
- Read 20000/30000 words so far
Current noun 'favorite'
Current noun 'warm'
Current noun 'good'
Current noun 'real'
Current noun 'easy'
Current noun 'rich'
Current noun 'guilt'
- Read 30000/30000 words so far


# 1. Determine approximately the top-10 words

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

In [10]:
def add_to_reservoir(reservoir, item, max_reservoir_size):
    # YOUR CODE HERE
    
    if len(reservoir) < max_reservoir_size:
        # If the reservoir is not full we add the item
        reservoir.append(item)
    else:
        random_index = random.randint(0, max_reservoir_size - 1)
        reservoir[random_index] = item

    
    assert(len(reservoir) <= max_reservoir_size)

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

In [15]:
def reservoir_sampling(filename, parts_of_speech, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []

    words_read = 0

    for word in read_by_parts_of_speech(filename, parts_of_speech, max_words=max_words, report_every=report_every):

            # YOUR CODE HERE
            
        words_read += 1

        if len(reservoir) < reservoir_size:
            reservoir.append(word)
        else:
            probability = reservoir_size / words_read
            if random.random() < probability:
                # replace a random element in the reservoir
                add_to_reservoir(reservoir, word, reservoir_size)

    return (words_read, reservoir)

In [16]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, [POS_ADJECTIVE], reservoir_size, max_words=30000, report_every=10000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Number of items seen    : 30001
Number of items sampled : 1500


In [17]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:20]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

51 good
40 little
29 last
27 sure
27 much
25 right
24 other
23 dead
22 real
20 sorry
19 bad
17 old
17 many
16 wrong
16 ready
16 great
16 big
15 next
15 new
14 first


<font size="+1" color="red">Replace this cell with your code to print the top items and their relative frequencies</font>

In [19]:
total_items = len(reservoir)

# Print absolute and relative frequencies
print("Top 20 most frequent items and their relative frequencies:")
for absolute_frequency, word in most_frequent_items:
    relative_frequency = (absolute_frequency / total_items) * 100
    print(f"{absolute_frequency} ({relative_frequency:.2f}%) {word}")


Top 20 most frequent items and their relative frequencies:
51 (3.40%) good
40 (2.67%) little
29 (1.93%) last
27 (1.80%) sure
27 (1.80%) much
25 (1.67%) right
24 (1.60%) other
23 (1.53%) dead
22 (1.47%) real
20 (1.33%) sorry
19 (1.27%) bad
17 (1.13%) old
17 (1.13%) many
16 (1.07%) wrong
16 (1.07%) ready
16 (1.07%) great
16 (1.07%) big
15 (1.00%) next
15 (1.00%) new
14 (0.93%) first


<font size="+1" color="red">Increase the max limit of words so that one pass takes about 2-3 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.</font>

In [39]:
max_words = 50000  # This value should be chosen so that the processing takes about 2-3 minutes

reservoir_sizes = [50, 100, 500, 1000]


# Loop over different  sizes
for reservoir_size in reservoir_sizes:
    print(f"\nReservoir size: {reservoir_size}")
    
    items_seen, reservoir = reservoir_sampling(INPUT_FILE, [POS_ADJECTIVE], reservoir_size, max_words=max_words, report_every=10000)
    
    freq = {}
    for item in reservoir:
        freq[item] = reservoir.count(item)
    
    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]
    
    # Print top 5 words 
    print("Top 5 words in the reservoir:")
    for absolute_frequency, word in most_frequent_items:
        estimated_frequency = absolute_frequency * max_words / reservoir_size
        print(f"Word: {word}, Absolute Frequency: {absolute_frequency}, Estimated Frequency: {estimated_frequency:.2f}")


Reservoir size: 50
- Read 10000/50000 words so far
- Read 20000/50000 words so far
- Read 30000/50000 words so far
- Read 40000/50000 words so far
- Read 50000/50000 words so far
Top 5 words in the reservoir:
Word: uh, Absolute Frequency: 2, Estimated Frequency: 2000.00
Word: real, Absolute Frequency: 2, Estimated Frequency: 2000.00
Word: many, Absolute Frequency: 2, Estimated Frequency: 2000.00
Word: last, Absolute Frequency: 2, Estimated Frequency: 2000.00
Word: hard, Absolute Frequency: 2, Estimated Frequency: 2000.00

Reservoir size: 100
- Read 10000/50000 words so far
- Read 20000/50000 words so far
- Read 30000/50000 words so far
- Read 40000/50000 words so far
- Read 50000/50000 words so far
Top 5 words in the reservoir:
Word: good, Absolute Frequency: 6, Estimated Frequency: 3000.00
Word: right, Absolute Frequency: 3, Estimated Frequency: 1500.00
Word: little, Absolute Frequency: 3, Estimated Frequency: 1500.00
Word: hundred, Absolute Frequency: 3, Estimated Frequency: 1500.00

<font size="+1" color="red">Replace this cell with a brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

Larger reservoirs (for example 1000) have more of the diversity, giving more stable top-3 results.

Size 1000 gives me back the same 3 words few times (good, little, other). But i suposse than even a bigger size would be better.

# 2. Determine approximately the distinct number of words

In [20]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [21]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

In [25]:
number_of_passes = 5
estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    hash_function = random_hash_function()

    max_trailing_zeroes = 0

    for word in read_by_parts_of_speech(INPUT_FILE, [POS_ADJECTIVE], max_words= 30000, report_every= 10000):
        
        # Calcular el valor del hash 
        hash_value = hash_function(word)
        
        # Contar los ceros 
        trailing_zeroes = count_trailing_zeroes(hash_value)

        max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

    # Estimar el número de palabras distintas 
    estimate = 2 ** max_trailing_zeroes
    
    
    
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))


- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 1: 2048 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 2: 4096 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 3: 4096 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 4: 16384 distinct words
- Read 10000/30000 words so far
- Read 20000/30000 words so far
- Read 30000/30000 words so far
Estimate on pass 5: 8192 distinct words


In [26]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 6963.2
* Median  of estimates: 4096.0


<font size="+1" color="red">Compute the median of average estimates in 3 separate runs of your algorithm; each run should do 10 passes over the file. Repeat this for nouns (POS_NOUN), adjectives (POS_ADJECTIVE), and verbs (POS_VERB). Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

In [30]:
number_of_passes = 10
estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    hash_function = random_hash_function()

    max_trailing_zeroes = 0

    for word in read_by_parts_of_speech(INPUT_FILE, [POS_ADJECTIVE], max_words= 10000, report_every= 10000):
        
        # Calcular el valor del hash 
        hash_value = hash_function(word)
        
        # Contar los ceros 
        trailing_zeroes = count_trailing_zeroes(hash_value)

        max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

    # Estimar el número de palabras distintas 
    estimate = 2 ** max_trailing_zeroes
    
    
    
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))


- Read 10000/10000 words so far
Estimate on pass 1: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 2: 8192 distinct words
- Read 10000/10000 words so far
Estimate on pass 3: 2048 distinct words
- Read 10000/10000 words so far
Estimate on pass 4: 2048 distinct words
- Read 10000/10000 words so far
Estimate on pass 5: 16384 distinct words
- Read 10000/10000 words so far
Estimate on pass 6: 16384 distinct words
- Read 10000/10000 words so far
Estimate on pass 7: 4096 distinct words
- Read 10000/10000 words so far
Estimate on pass 8: 16384 distinct words
- Read 10000/10000 words so far
Estimate on pass 9: 4096 distinct words
- Read 10000/10000 words so far
Estimate on pass 10: 32768 distinct words


In [31]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 10342.4
* Median  of estimates: 6144.0


In [32]:
number_of_passes = 10
estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    hash_function = random_hash_function()

    max_trailing_zeroes = 0

    for word in read_by_parts_of_speech(INPUT_FILE, [POS_NOUN], max_words= 10000, report_every= 10000):
        
        # Calcular el valor del hash 
        hash_value = hash_function(word)
        
        # Contar los ceros 
        trailing_zeroes = count_trailing_zeroes(hash_value)

        max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

    # Estimar el número de palabras distintas 
    estimate = 2 ** max_trailing_zeroes
    
    
    
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))


- Read 10000/10000 words so far
Estimate on pass 1: 262144 distinct words
- Read 10000/10000 words so far
Estimate on pass 2: 2048 distinct words
- Read 10000/10000 words so far
Estimate on pass 3: 8192 distinct words
- Read 10000/10000 words so far
Estimate on pass 4: 8192 distinct words
- Read 10000/10000 words so far
Estimate on pass 5: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 6: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 7: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 8: 4096 distinct words
- Read 10000/10000 words so far
Estimate on pass 9: 16384 distinct words
- Read 10000/10000 words so far
Estimate on pass 10: 8192 distinct words


In [33]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 31232.0
* Median  of estimates: 6144.0


In [34]:
number_of_passes = 10
estimates = []

for i in range(number_of_passes):
    # YOUR_CODE_HERE: read the file and generate an estimate
    hash_function = random_hash_function()

    max_trailing_zeroes = 0

    for word in read_by_parts_of_speech(INPUT_FILE, [POS_VERB], max_words= 10000, report_every= 10000):
        
        # Calcular el valor del hash 
        hash_value = hash_function(word)
        
        # Contar los ceros 
        trailing_zeroes = count_trailing_zeroes(hash_value)

        max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)

    # Estimar el número de palabras distintas 
    estimate = 2 ** max_trailing_zeroes
    
    
    
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))


- Read 10000/10000 words so far
Estimate on pass 1: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 2: 512 distinct words
- Read 10000/10000 words so far
Estimate on pass 3: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 4: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 5: 512 distinct words
- Read 10000/10000 words so far
Estimate on pass 6: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 7: 2048 distinct words
- Read 10000/10000 words so far
Estimate on pass 8: 1024 distinct words
- Read 10000/10000 words so far
Estimate on pass 9: 256 distinct words
- Read 10000/10000 words so far
Estimate on pass 10: 512 distinct words


In [35]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 896.0
* Median  of estimates: 1024.0


<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>